```bash						
	NAME	DM [pc cm**-3]	S/N (AMBER)	RA	DEC	POINTING NAME	CB Found
							
1	FRB 190709	663.4	15.4	01h36m06.7s	+31d51m22.8s	3C48drift2732	10
2	FRB 190903	663.8	10.8	01h32m43.2805s	+33d04m48.9206s		4
3	FRB 190925	956.7	12.9	01h41m49.s	+30d59m24.4s		7
4	FRB 191020	465	13.2	20h30m52 	+61d58m47s	T2029+6307	5
5	FRB 191108	587	60	01h33m57.35s	+31d45m38s	FRBfield	21
6	FRB 191109	531	13	20:33:51	61:46:30	FRB191020	18
```

```bash
python2 /home/arts/ARTS-obs/external/arts-analysis/triggers.py 
--rficlean
--sig_thresh_local 5
--time_limit 9000.0
--descending_snr 
--beamno 00
--dm_min 29.360000000000003
--dm_max 5000
--sig_thresh 10 
--ndm 64
--save_data concat
--nfreq_plot 64
--ntime_plot 64
--cmap viridis 
--outdir=/data2/output/20191129/2019-11-29-09:45:00.T1703+6759/triggers 
--clean_type perchannel
--synthesized_beams
--sbmin 36
--sbmax 70 
--central_freq 1370 
/data2/output/20191129/2019-11-29-09:45:00.T1703+6759/filterbank/CB00 
/data2/output/20191129/2019-11-29-09:45:00.T1703+6759/amber/CB00.trigger
```

In [ ]:
from tools import *

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

# from matplotlib import rc
import matplotlib.pyplot as plt
from matplotlib.figure import figaspect
from matplotlib import rc
import datetime
from matplotlib.backends.backend_pdf import PdfPages

from astropy.visualization import hist as ahist

from . import *

import numpy as np
from scipy import stats
import scipy


rc('font', size=12)
rc('axes', titlesize=16)
rc('axes', labelsize=12)

In [ ]:
def abline(slope, intercept):
    """Plot a line from slope and intercept"""
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '--')
    
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [ ]:
def read_singlepulse_dev(fn, max_rows=None, beam=None):
    """ Read in text file containing single-pulse 
    candidates. Allowed formats are:
    .singlepulse = PRESTO output
    .txt = injection pipeline output
    .trigger = AMBER output 
    .cand = heimdall output 

    max_rows sets the maximum number of 
    rows to read from textfile 
    beam is the beam number to pick in case of .trigger files
    """

    if fn.split('.')[-1] in ('singlepulse', 'txt'):
        A = np.genfromtxt(fn, max_rows=max_rows)

        if len(A.shape) == 1:
            A = A[None]

        dm, sig, tt, downsample = A[:,0], A[:,1], A[:,2], A[:,4]
    elif fn.split('.')[-1] == 'trigger':
        A = np.genfromtxt(fn, max_rows=max_rows)

        if len(A.shape) == 1:
            A = A[None]

        # Check if amber has compacted, in which case 
        # there are two extra rows
        if len(A[0]) > 7:
            if len(A[0])==8:
                # beam batch sample integration_step compacted_integration_steps time DM compacted_DMs SNR
                beamno, dm, sig, tt, downsample = A[:, 0], A[:, -3], A[:, -1], A[:, -4], A[:, 3]
            elif len(A[0])==10:
                beamno, dm, sig, tt, downsample = A[:, 0], A[:, -3], A[:, -1], A[:, -5], A[:, 3]
            else:
                print("Error: DO NOT RECOGNIZE COLUMNS OF .trigger FILE")
                return 
        else:
            # beam batch sample integration_step time DM SNR
            beamno, dm, sig, tt, downsample = A[:, 0], A[:,-2], A[:,-1], A[:, -3], A[:, 3]
        
        if beam is not None and beam != 'all':
                # pick only the specified beam
                dm = dm[beamno.astype(int) == beam]
                sig = sig[beamno.astype(int) == beam]
                tt = tt[beamno.astype(int) == beam]
                downsample = downsample[beamno.astype(int) == beam]

    elif fn.split('.')[-1] == 'cand':
        A = np.genfromtxt(fn, max_rows=max_rows)

        if len(A.shape) == 1:
            A = A[None]
        
        # SNR sample_no time log_2_width DM_trial DM Members first_samp last_samp
        dm, sig, tt, log_2_downsample = A[:,5], A[:,0], A[:, 2], A[:, 3]
        downsample = 2**log_2_downsample
        try:
            beamno = A[:, 9]
            return dm, sig, tt, downsample, beamno
        except:
            pass
    elif fn.split('.')[-1]=='fredda':
        A = np.genfromtxt(fn, max_rows=max_rows)

        if len(A.shape)==1:
            A = A[None]
        
        dm, sig, tt, downsample = A[:,5], A[:,0], A[:, 2], A[:, 3]
    else:
        print("Didn't recognize singlepulse file")
        return 

    if len(A) == 0:
        if beam == 'all':
            return 0, 0, 0, 0, 0
        else:
            return 0, 0, 0, 0

    if beam == 'all':
        return dm, sig, tt, downsample, beamno
    else:
        return dm, sig, tt, downsample



In [ ]:
def get_triggers_dev(fn, sig_thresh=5.0, dm_min=0, dm_max=np.inf, 
                 t_window=0.5, max_rows=None, t_max=np.inf,
                 sig_max=np.inf, dt=2*40.96, delta_nu_MHz=300./1536, 
                 nu_GHz=1.4, fnout=False, tab=None, read_beam=False, dm_width_filter=False):
    """ Get brightest trigger in each 10s chunk.

    Parameters
    ----------
    fn : str 
        filename with triggers (.npy, .singlepulse, .trigger)
    sig_thresh : float
        min S/N to include
    dm_min : 
        minimum dispersion measure to allow 
    dm_max : 
        maximum dispersion measure to allow 
    t_window : float 
        Size of each time window in seconds
    max_rows : 
        Only read this many rows from raw trigger file 
    fnout : str 
        name of text file to save clustered triggers to 
    tab : int
        which TAB to process (0 for IAB)
    read_beam: bool
        read and return beam number (default: False)
        all beams are read if this is true

    Returns
    -------
    sig_cut : ndarray
        S/N array of brightest trigger in each DM/T window 
    dm_cut : ndarray
        DMs of brightest trigger in each DM/T window 
    tt_cut : ndarray
        Arrival times of brightest trigger in each DM/T window 
    ds_cut : ndarray 
        downsample factor array of brightest trigger in each DM/T window
    beam_cut: ndarray
        beam array of brightest trigger in each DM/T windows (only if read_beam is True)
    """
    if tab is not None:
        beam_amber = tab
    elif read_beam:
        beam_amber = 'all'
    else:
        beam_amber = None

    if type(fn) == str:
        if read_beam:
            dm, sig, tt, downsample, beam = read_singlepulse_dev(fn, max_rows=max_rows, beam=beam_amber)[:5]
        else:
            dm, sig, tt, downsample = read_singlepulse_dev(fn, max_rows=max_rows, beam=beam_amber)[:4]
    elif type(fn) == np.ndarray:
        dm, sig, tt, downsample = fn[:, 0], fn[:, 1], fn[:, 2], fn[:, 3]
    else:
        print("Wrong input type. Expected string or ndarray")
        if read_beam:
            return [], [], [], [], [], []
        else:
            return [], [], [], [], []

    ntrig_orig = len(dm)

    bad_sig_ind = np.where((sig < sig_thresh) | (sig > sig_max))[0]
    sig = np.delete(sig, bad_sig_ind)
    tt = np.delete(tt, bad_sig_ind)
    dm = np.delete(dm, bad_sig_ind)
    downsample = np.delete(downsample, bad_sig_ind)
    sig_cut, dm_cut, tt_cut, ds_cut = [], [], [], []
    if read_beam:
        beam = np.delete(beam, bad_sig_ind)
        beam_cut = []

    if len(tt) == 0:
        print("Returning None: time array is empty")
        return 

    tduration = tt.max() - tt.min()
    ntime = int(tduration / t_window)

    # Make dm windows between 90% of the lowest trigger and 
    # 10% of the largest trigger
    if dm_min == 0:
        dm_min = 0.9*dm.min()
    if dm_max > 1.1*dm.max():
        dm_max = 1.1*dm.max()

    # Can either do the DM selection here, or after the loop
#    dm_list = dm_range(dm_max, dm_min=dm_min)
    dm_list = dm_range(1.1*dm.max(), dm_min=0.9*dm.min())

    print("\nGrouping in window of %.2f sec" % np.round(t_window,2))
    print("DMs:", dm_list)

    tt_start = tt.min() - .5*t_window
    ind_full = []

    # might wanna make this a search in (dm,t,width) cubes
    for dms in dm_list:
        for ii in range(ntime + 2):
            try:    
                # step through windows of t_window seconds, starting from tt.min()
                # and find max S/N trigger in each DM/time box
                t0, tm = t_window*ii + tt_start, t_window*(ii+1) + tt_start
                ind = np.where((dm<dms[1]) & (dm>dms[0]) & (tt<tm) & (tt>t0))[0] 
                
                if len(ind) > 0:
                    plt.plot(sig[ind], label=ii)
                
                ind_maxsnr = ind[np.argmax(sig[ind])]
                sig_cut.append(sig[ind_maxsnr])
                dm_cut.append(dm[ind_maxsnr])
                tt_cut.append(tt[ind_maxsnr])
                ds_cut.append(downsample[ind_maxsnr])
                if read_beam:
                    beam_cut.append(beam[ind_maxsnr])
                ind_full.append(ind_maxsnr)
            except:
                continue

    ind_full = np.array(ind_full)
    dm_cut = np.array(dm_cut)
    # now remove the low DM candidates
    tt_cut = np.array(tt_cut).astype(np.float)
    ind = np.where((dm_cut >= dm_min) & (dm_cut <= dm_max) & (tt_cut < t_max))[0]

    dm_cut = dm_cut[ind]
    ind_full = ind_full[ind]
    sig_cut = np.array(sig_cut)[ind]
    tt_cut = tt_cut[ind]
    ds_cut = np.array(ds_cut)[ind]
    if read_beam:
        beam_cut = np.array(beam_cut)[ind]

    ntrig_group = len(dm_cut)

    print("Grouped down to %d triggers from %d\n" % (ntrig_group, ntrig_orig))

    rm_ii = []

    if dm_width_filter:
        for ii in range(len(ds_cut)):        
            tdm = 8.3 * delta_nu_MHz / nu_GHz**3 * dm_cut[ii] # microseconds#

            if ds_cut[ii]*dt < (0.5*(dt**2 + tdm**2)**0.5):
                rm_ii.append(ii)

    dm_cut = np.delete(dm_cut, rm_ii)
    tt_cut = np.delete(tt_cut, rm_ii)
    sig_cut = np.delete(sig_cut, rm_ii)
    ds_cut = np.delete(ds_cut, rm_ii)
    if read_beam:
        beam_cut = np.delete(beam_cut, rm_ii)
    ind_full = np.delete(ind_full, rm_ii)

    if fnout:
        if read_beam:
            clustered_arr = np.concatenate([sig_cut, dm_cut, tt_cut, ds_cut, beam_cut, ind_full])
            clustered_arr = clustered_arr.reshape(6, -1)
        else:
            clustered_arr = np.concatenate([sig_cut, dm_cut, tt_cut, ds_cut, ind_full])
            clustered_arr = clustered_arr.reshape(5, -1)
        np.savetxt(fnout, clustered_arr) 

    if read_beam:
        return sig_cut, dm_cut, tt_cut, ds_cut, beam_cut, ind_full
    else:
        return sig_cut, dm_cut, tt_cut, ds_cut, ind_full

# Load / parse data

In [ ]:
sigs

In [ ]:
iiii=30

plt.plot(beams[iiii], (sigs[iiii]-np.max(sigs[iiii]))/np.median(sigs[iiii]))

In [ ]:
plt.plot(beams[30], dms_[30])
abline(0, np.median(dms_[30]))
print (np.median(dms_[30]))

In [ ]:
iiii = 70
iiii = 40

# __s = 1+(sigs[iiii]-np.max(sigs[iiii]))/(np.max(sigs[iiii])-np.min(sigs[iiii]))
__s = (sigs[iiii]-np.median(sigs[iiii]))/(np.std(sigs[iiii]))

plt.plot(beams[iiii], __s)
abline(0, np.max(__s) - np.std(__s))

np.count_nonzero(
    np.isclose(
        [__s], 
        [np.max(__s)], 
        atol=np.std(__s)
    )
)

for beam, sb in zip(beams[iiii], sigs[iiii]):
    print (beam, sb)


In [ ]:
plt.plot(beams[iiii], dms_[iiii])
abline(0, np.median(dms_[iiii]))
print (np.median(dms_[iiii]))

In [ ]:
ranges = []
peaks = []

# Find FRB 191108:
for i, snr in enumerate(sigs):
    snr = np.asarray(snr)
    ranges.append(snr.max()-snr.min())
#     _s = 1+(snr-np.max(snr))/(np.max(snr)-np.min(snr))
    _s = (snr-np.median(snr))/(np.std(snr))
    peaks.append(
        np.count_nonzero(
            np.isclose(
                [_s], 
                [np.max(_s)], 
                atol=np.std(_s)
            )
        )
    )

In [ ]:
peaks[40]

In [ ]:
plt.plot(ranges)
plt.xlabel('Trigger X')
plt.ylabel('SNR range (max-min)')

In [ ]:
print (len(peaks))
plt.hist(peaks, bins=np.asarray(range(len(peaks)+1)))
# plt.xlabel('Trigger X')
# plt.ylabel('N peaks')

In [ ]:
for i, r in  enumerate(ranges):
    print ("%d\t%.5f" % (i, r))

In [ ]:
np.isclose?

In [ ]:
np.isclose(([(sigs[iiii]-np.max(sigs[iiii]))/np.median(sigs[iiii])]), [0], atol=0.1)[0]

In [ ]:
plt.plot(_s)

In [ ]:
np.count_nonzero(
    np.isclose(
        [_s], 
        [1], 
        atol=0.05
    )
)

In [ ]:
plt.hist?

In [ ]:
[0]

In [ ]:
plt.plot(beams[iiii], sigs[iiii])

In [ ]:
npoints = 5

print (sigs[iiii][np.argsort(sigs[iiii])[-npoints:]])

In [ ]:
np.argsort(sigs[iiii])[-5:]

In [ ]:
iiii = 40
ntop = 10

_sorted = np.argsort(sigs[iiii])[-ntop:]

top_sigs = np.asarray(sigs[iiii])[_sorted]
top_beams = np.asarray(beams[iiii])[_sorted]

slope, intercept, rvalue, pvalue, stderr = stats.linregress(top_beams, top_sigs)

print("slope", slope, )
print("intercept", intercept, )
print("rvalue", rvalue, )
print("pvalue", pvalue, )
print("stderr", stderr)

plt.plot(beams[iiii], sigs[iiii])
abline(slope, intercept)

In [ ]:
iiii = 22

_sorted = np.argsort(sigs[iiii])[-ntop:]
top_sigs = np.asarray(sigs[iiii])[_sorted]
top_beams = np.asarray(beams[iiii])[_sorted]

slope, intercept, rvalue, pvalue, stderr = stats.linregress(top_beams, top_sigs)
 
print("slope", slope, )
print("intercept", intercept, )
print("rvalue", rvalue, )
print("pvalue", pvalue, )
print("stderr", stderr)

plt.scatter(beams[iiii], sigs[iiii])
abline(slope, intercept)

In [ ]:
plt.plot(a)

In [ ]:
plt.hist(sigs[40], len(sigs[40])//2)
plt.xlabel('SNR')

In [ ]:
plt.hist(sigs[20])

In [ ]:
plt.hist(sigs[20], len(sigs[20])//2)

In [ ]:
plt.hist(sigs[45])

In [ ]:
plt.hist(sigs[127])

In [ ]:
plt.hist(sigs[127])

In [ ]:
sigs[26]

In [ ]:
plt.plot(np.absolute(np.fft.fft(sigs[40])))



In [ ]:
plt.plot(np.(sigs[17]))

In [ ]:
len(sigs[40])

In [ ]:
plt.plot(beams[40], sigs[40])

In [ ]:
arr = np.NAN(71)
arr[np.array(beams[40]).astype(int)] = sigs[40]

In [ ]:
plt.plot(a)

In [ ]:
plt.plot(np.absolute(np.fft.fft(a)))

In [ ]:
for i in range(0, 71, 11):
    print (i, np.absolute(np.fft.fft(arr))[i])

In [ ]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

idx = 40
# idx = 101 

print (beams[idx])
print (sigs[idx])

a = np.empty(71)
a[:] = 8
a[np.array(beams[idx]).astype(int)] = sigs[idx]
nans, x= nan_helper(a)
a[nans]= np.interp(x(nans), x(~nans), a[~nans])
# if 11 in scipy.signal.find_peaks(np.absolute(np.fft.fft(a))):
#     print ('Yay')
fig, ax = plt.subplots(2,1)

ax[0].plot(a)
ax[0].set_xlabel('SB')
ax[0].set_ylabel('SNR')
ax[1].plot(np.absolute(np.fft.fft(a)[1:]))
ax[1].set_xlabel('Freq')
ax[1].set_ylabel('Amplitude')
plt.tight_layout()

fourier_dom = np.absolute(np.fft.fft(a))

fourier_dom.shape



# for i in range(0, 71, 11):
#     print (i, np.absolute(np.fft.fft(a))[i])
    
# print ()

# for i, j in enumerate(np.absolute(np.fft.fft(a))):
#     print (i, j)

In [ ]:
fourier_dom.shape

In [ ]:
fourier_dom[11]/fourier_dom[30:40].mean()

In [ ]:
_idx = 2
a = np.empty(71)
a[:] = np.nan
a[np.array(beams[_idx]).astype(int)] = sigs[_idx]
nans, x= nan_helper(a)
a[nans]= np.interp(x(nans), x(~nans), a[~nans])
plt.plot(np.absolute(np.fft.fft(a)))

fourier_dom = np.absolute(np.fft.fft(a))

# for i in range(0, 71, 11):
#     print (i, np.absolute(np.fft.fft(a))[i])
    
# print ()

# for i, j in enumerate(np.absolute(np.fft.fft(a))):
#     print (i, j)